In [4]:
import numpy as np
from ase.io import read, write
from ase.build import make_supercell

def hexagonal_to_rectangular(input_file='POSCAR1.vasp', output_file='POSCAR_rectangular.vasp'):
    # Read the hexagonal structure from POSCAR1.vasp
    try:
        atoms = read(input_file, format='vasp')
        print(f"Original cell (hexagonal):\n{atoms.cell}")
    except FileNotFoundError:
        print(f"Error: {input_file} not found.")
        return
    
    # Original hexagonal lattice vectors
    cell = atoms.cell
    a1 = cell[0]  # e.g., [a, 0, 0]
    a2 = cell[1]  # e.g., [a/2, a√3/2, 0]
    a3 = cell[2]  # e.g., [0, 0, c]

    # Define the transformation matrix based on (1,1) and (-1,1)
    transform_matrix = np.array([
        [1, 1, 0],  # a' = a1 + a2
        [-1, 1, 0], # b' = -a1 + a2
        [0, 0, 1]   # c' = a3 (unchanged)
    ])

    # Create the supercell
    rectangular_atoms = make_supercell(atoms, transform_matrix)

    # Verify the new cell is rectangular
    new_cell = rectangular_atoms.cell
    print(f"Transformed cell (rectangular):\n{new_cell}")
    
    # Check orthogonality (dot product of a and b should be ~0)
    dot_product = np.dot(new_cell[0], new_cell[1])
    print(f"Dot product of new a and b: {dot_product:.6f} (should be close to 0 for rectangular)")

    # Write the rectangular structure to a new POSCAR file
    write(output_file, rectangular_atoms, format='vasp', vasp5=True, direct=False)
    print(f"Rectangular structure saved to {output_file}")

if __name__ == '__main__':
    hexagonal_to_rectangular()

Original cell (hexagonal):
Cell([[2.46729, 0.0, 0.0], [1.23365, 2.13674, 0.0], [0.0, 0.0, 7.80307]])
Transformed cell (rectangular):
Cell([[3.70094, 2.13674, 0.0], [-1.2336400000000003, 2.13674, 0.0], [0.0, 0.0, 7.80307]])
Dot product of new a and b: 0.000030 (should be close to 0 for rectangular)
Rectangular structure saved to POSCAR_rectangular.vasp
